In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from openpyxl.styles import Font

path = kagglehub.dataset_download("bejopamungkas/transaksi-pembelian-penjualan-sembako")

df_pemasukan_barang = pd.read_csv(os.path.join(path, "pemasukan barang.csv"))
df_penjualan_barang = pd.read_csv(os.path.join(path, "penjualan barang.csv"))

c:\Users\51206614\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Summary Keseluruhan

In [2]:
jml_barang_masuk = df_pemasukan_barang.groupby([pd.Grouper(key='nama.barang')]).sum().kuantum
jml_barang_dijual = df_penjualan_barang.groupby([pd.Grouper(key='nama.barang')]).sum().kuantum
total_pemasukan = df_penjualan_barang.groupby([pd.Grouper(key='nama.barang')]).sum().nominal
sisa_stock = jml_barang_masuk - jml_barang_dijual

##### Reporting

In [3]:
laporan_stok = pd.DataFrame({
    "Barang": jml_barang_masuk.index,
    "Total Masuk": jml_barang_masuk.values,
    "Total Terjual": jml_barang_dijual.values,
    "Sisa Stok": sisa_stock.values
})

def format_rupiah(x):
    return f"Rp {x:,.0f}".replace(",", ".")

pembeli = df_penjualan_barang.groupby('nama.pembeli').sum(numeric_only=True)['nominal']
laporan_pembeli = pembeli.sort_values(ascending=False).head(10).apply(format_rupiah).reset_index()
laporan_pembeli.columns = ["Nama Pembeli", "Total Pembelian"]

laporan_penjualan_harian = df_penjualan_barang.groupby("tanggal").sum(numeric_only=True)[["nominal", "kuantum"]].reset_index()

file_excel = "Laporan_Sembako.xlsx"

with pd.ExcelWriter(file_excel, engine="openpyxl") as writer:
    laporan_stok.to_excel(writer, sheet_name="Stok Barang", index=False)
    laporan_pembeli.to_excel(writer, sheet_name="Top 10 Pembeli", index=False)
    laporan_penjualan_harian.to_excel(writer, sheet_name="Penjualan Harian", index=False)

wb = load_workbook(file_excel)
for sheet in wb.sheetnames:
    ws = wb[sheet]
    ws["A1"].font = Font(bold=True, color="0000FF") 

wb.save(file_excel)

print(f"Laporan berhasil diekspor ke {file_excel} 🎉")


Laporan berhasil diekspor ke Laporan_Sembako.xlsx 🎉


### Peringkat Pembeli

In [4]:
pembeli.head(10)

nama.pembeli
ANAPSA                                      43926000.0
ANDY                                        26105700.0
HOLIDAY MART                                59500000.0
Harya Sari                                  25094800.0
KARUNIA                                     28558700.0
KOPERASI BRI MELAWI                         32422400.0
Langkah Jaya                                 8968000.0
MEKAR SARI                                  38093800.0
PT. BANK RAKYAT INDONESIA (PERSERO) Tbk.    16185000.0
RUDIMAS                                      7380000.0
Name: nominal, dtype: float64

### Summary Per-hari

In [ ]:
df_pemasukan_barang_group = df_pemasukan_barang.groupby(['tanggal', 'nama.barang'], as_index=False).sum()
df_penjualan_barang_group = df_penjualan_barang.groupby(['tanggal', 'nama.barang','nama.pembeli'], as_index=False).sum()

### Ploting

In [ ]:
def plotting(nama_toko):
    sns.set(style="whitegrid")
    
    data_pemasukan = df_pemasukan_barang.groupby(['tanggal', 'nama.barang'], as_index=False).sum()
    data_penjualan = df_penjualan_barang.groupby(['tanggal', 'nama.barang'], as_index=False).sum()
    
    data_pemasukan['tanggal'] = pd.to_datetime(data_pemasukan['tanggal'])
    data_penjualan['tanggal'] = pd.to_datetime(data_penjualan['tanggal'])
    
    data_pemasukan_filtered = data_pemasukan[data_pemasukan['nama.barang'] == nama_toko.upper()]
    data_penjualan_filtered = data_penjualan[data_penjualan['nama.barang'] == nama_toko.upper()]
    
    plt.figure(figsize=(12, 7))
    
    plt.plot(data_pemasukan_filtered['tanggal'], data_pemasukan_filtered['kuantum'],
             linestyle='-', linewidth=2, markersize=8, label='Pemasukan')
    
    plt.plot(data_penjualan_filtered['tanggal'], data_penjualan_filtered['kuantum'],
             linestyle='-', linewidth=2, markersize=8, label='Penjualan')
    
    plt.xlabel('Tanggal', fontsize=14)
    plt.ylabel('Kuantum', fontsize=14)
    plt.title(f"Tren Kuantum Barang {nama_toko.upper()} (Pemasukan vs Penjualan)", fontsize=16)
    plt.xticks(rotation=45)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plotting('beras')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plotting(nama_toko):
    sns.set(style="whitegrid")

    # Grouping data penjualan berdasarkan tanggal dan nama pembeli
    data_penjualan = df_penjualan_barang.groupby(['tanggal', 'nama.pembeli'], as_index=False).sum()

    # Konversi tanggal menjadi datetime
    data_penjualan['tanggal'] = pd.to_datetime(data_penjualan['tanggal'])

    # Filter hanya untuk toko tertentu
    data_penjualan_filtered = data_penjualan[data_penjualan['nama.pembeli'] == nama_toko.upper()]

    # Jika data kosong, beri warning
    if data_penjualan_filtered.empty:
        print(f"Tidak ada data untuk toko {nama_toko}")
        return

    plt.figure(figsize=(10, 5))

    # Menggunakan Seaborn untuk lineplot yang lebih smooth
    sns.lineplot(x=data_penjualan_filtered['tanggal'], 
                 y=data_penjualan_filtered['nominal'] / 1_000_000,  # Menggunakan satuan juta
                 marker='o', linewidth=2, label="Penjualan")

    # Format plot
    plt.xlabel('Tanggal', fontsize=12)
    plt.ylabel('Nominal (Juta Rupiah)', fontsize=12)
    plt.title(f"Tren Penjualan {nama_toko.upper()} dari Waktu ke Waktu", fontsize=14)
    plt.xticks(rotation=45)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Tampilkan plot
    plt.show()
